In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import random
import plotly.express as px
import time
import seaborn as sns
sns.set_theme('talk', palette=sns.color_palette('Set2'))
pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 150

In [2]:
infile = open('..//3_k-means_clustering//apartment_data_frame_clustered','rb')
frame = pkl.load(infile)
infile.close()

In [3]:
frame.head()

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url,price_of_sqm,cluster
0,Kołobrzeg,kołobrzeski,zachodniopomorskie,54.175917,15.583267,aftermarket,sprzedaz,75.85,2,4,4,1970.0,850000.0,https://gratka.pl/nieruchomosci/mieszkanie-kolobrzeg/ob/25201643,11206.328279,14
1,Bydgoszcz,NaN,kujawsko-pomorskie,53.141840,17.952490,aftermarket,sprzedaz,64.00,3,2,3,2015.0,539000.0,https://gratka.pl/nieruchomosci/mieszkanie-bydgoszcz/ob/26514629,8421.875000,292
2,Wrocław,Stare Miasto,dolnośląskie,51.101884,17.027313,aftermarket,sprzedaz,122.00,4,3,4,1920.0,849000.0,https://gratka.pl/nieruchomosci/mieszkanie-wroclaw-stare-miasto-ul-michala-baluckiego/ob/25652501,6959.016393,397
3,Toruń,NaN,kujawsko-pomorskie,53.026200,18.590500,aftermarket,sprzedaz,63.50,3,4,3,1993.0,459000.0,https://gratka.pl/nieruchomosci/mieszkanie-torun-ul-wodociagowa/ob/26134151,7228.346457,452
4,Swarzędz,poznański,wielkopolskie,52.416900,17.083000,aftermarket,sprzedaz,53.00,3,3,4,1981.0,359000.0,https://gratka.pl/nieruchomosci/mieszkanie-swarzedz/ob/25317957,6773.584906,121


In [4]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49263 entries, 0 to 49262
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            49263 non-null  object 
 1   district        41912 non-null  object 
 2   voivodeship     49263 non-null  object 
 3   localization_y  49263 non-null  float64
 4   localization_x  49263 non-null  float64
 5   market          49263 non-null  object 
 6   offer_type      49263 non-null  object 
 7   area            49263 non-null  float64
 8   rooms           49263 non-null  int32  
 9   floor           49263 non-null  int32  
 10  floors          49263 non-null  int32  
 11  build_yr        49263 non-null  float64
 12  price           49263 non-null  float64
 13  url             49263 non-null  object 
 14  price_of_sqm    49263 non-null  float64
 15  cluster         49263 non-null  int32  
dtypes: float64(6), int32(4), object(6)
memory usage: 5.3+ MB


In [5]:
to_drop = ['city','district','voivodeship','localization_y','localization_x','price_of_sqm','url','offer_type']
frame.drop(to_drop, axis=1, inplace=True)

## creating dummy variables

In [6]:
for column in ['cluster','rooms','floor','floors']:
    frame[column] = frame[column].astype('str')

In [7]:
frame = pd.get_dummies(frame, drop_first=True)

In [8]:
frame

,area,build_yr,price,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,floor_1,floor_10,floor_11,floor_12,...,cluster_86,cluster_87,cluster_88,cluster_89,cluster_9,cluster_90,cluster_91,cluster_92,cluster_93,cluster_94,cluster_95,cluster_96,cluster_97,cluster_98,cluster_99
0,75.85,1970.0,850000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,64.00,2015.0,539000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,122.00,1920.0,849000.0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,63.50,1993.0,459000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,53.00,1981.0,359000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49258,39.50,1972.0,630000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49259,30.50,1960.0,212000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49260,51.00,1960.0,740000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49261,54.64,1978.0,500000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## train test split

In [9]:
# from sklearn.model_selection import ShuffleSplit

# X = frame.drop('price',axis=1)
# y = frame.price

# ss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# train_index, test_index = next(ss.split(X))

# X_train = X.loc[train_index,:]
# y_train = y.loc[train_index]

# X_test = X.loc[test_index,:]
# y_test = y.loc[test_index]

In [10]:
from sklearn.model_selection import train_test_split

X = frame.drop('price',axis=1)
y = frame.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## data save

In [11]:
outfile = open('X_train','wb')
pkl.dump(X_train, outfile)
outfile.close()

outfile = open('X_test','wb')
pkl.dump(X_test, outfile)
outfile.close()

outfile = open('y_train','wb')
pkl.dump(y_train, outfile)
outfile.close()

outfile = open('y_test','wb')
pkl.dump(y_test, outfile)
outfile.close()